In [6]:
!pip install progressbar
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [0]:
%tensorflow_version 1.x
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
import sys
sys.path.append('/content/drive/My Drive/Colab_Work/PPM/io')
from hdf5datasetwriter import HDF5DatasetWriter
from imutils import paths
import numpy as np
import progressbar
import random
import os

In [0]:
bs = 32
buff_size=1000

In [9]:
db_name = input('Type your DB name here: ')
print(db_name)
outfile = db_name+'_features.hdf5'
print(outfile)

Type your DB name here: animals
animals
animals_features.hdf5


In [12]:
print("[INFO] loading images...")
%cd /content/drive/My\ Drive/Colab_Work
imagePaths = list(paths.list_images(db_name))
random.shuffle(imagePaths)

labels = [p.split(os.path.sep)[-2] for p in imagePaths]
le = LabelEncoder()
labels = le.fit_transform(labels)

print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)

dataset = HDF5DatasetWriter((len(imagePaths), 512 * 7 * 7), 
    outfile, dataKey="features", bufSize=buff_size)
dataset.storeClassLabels(le.classes_)

widgets = ["Extracting Features: ", progressbar.Percentage(), " ",
        progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagePaths), widgets=widgets).start()
out = display(progress(0, 100), display_id=True)
for i in np.arange(0, len(imagePaths), bs):
    batchPaths = imagePaths[i:i+bs]
    batchLabels = labels[i:i+bs]
    batchImages = []
    for (j, imagePath) in enumerate(batchPaths):
        image = load_img(imagePath, target_size=(224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)
        batchImages.append(image)
    batchImages = np.vstack(batchImages)
    features = model.predict(batchImages, batch_size=bs)
    features = features.reshape((features.shape[0], 512 * 7 * 7))
    dataset.add(features,  batchLabels)
    out.update(progress(i, len(imagePaths)-1))
    #pbar.update(i)

dataset.close()
#pbar.finish()

[INFO] loading images...
/content/drive/My Drive/Colab_Work
[INFO] loading network...
